In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
base_path = "/content/drive/MyDrive/SER_yt"
import os
os.makedirs(base_path, exist_ok=True)

In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00


In [4]:
import gradio as gr
import librosa
import numpy as np
import torch
import torch.nn.functional as F
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification

In [5]:
#Load your model and processor
model_path = "/content/drive/MyDrive/SER_yt/my_saved_model"
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
processor = Wav2Vec2Processor.from_pretrained(model_path)
model.eval()


Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)


In [6]:
# Label mapping
id2label = {
    0: "happy",
    1: "neutral",
    2: "sad",
    3: "fear",
    4: "disgust",
    5: "ps",
    6: "angry"
}


In [24]:
def predict_emotion_gradio(audio_input):
    try:
        # Check if the input is a tuple (mic recording): (sr, numpy_array)
        if isinstance(audio_input, tuple):
            sr, audio_np = audio_input
            audio_np = audio_np.astype(np.float32)

        # Else it's a file path (upload)
        else:
            audio_np, sr = librosa.load(audio_input, sr=None)
            audio_np = audio_np.astype(np.float32)

        # Resample if needed
        if sr != 16000:
            audio_np = librosa.resample(audio_np, orig_sr=sr, target_sr=16000)
            sr = 16000

        # Pad or trim to 2 seconds (32000 samples)
        audio_np = audio_np[:32000] if len(audio_np) > 32000 else np.pad(audio_np, (0, 32000 - len(audio_np)), mode='constant')

        # Process input
        inputs = processor(audio_np, sampling_rate=sr, return_tensors="pt")

        with torch.no_grad():
            logits = model(**inputs).logits
            probs = F.softmax(logits, dim=-1)

        predicted_id = torch.argmax(probs, dim=-1).item()
        confidence = torch.max(probs).item()

        return f"{id2label[predicted_id]} (Confidence: {round(confidence * 100, 2)}%)"

    except Exception as e:
        return f"Error: {str(e)}"


In [28]:
interface = gr.Interface(
    fn=predict_emotion_gradio,
    inputs=gr.Audio(type="numpy", label="Upload or Record Audio"),
    outputs="text",
    title="Speech Emotion Recognition",
    description="Upload or record a .wav audio and get the predicted emotion"
)



In [ ]:
interface.launch(share=True, debug=True)